In [0]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('./mmnist/data', one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mmnist/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mmnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mmnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mmnist/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [0]:
n_input = 28 *28
n_noise = 128
n_hidden = 256

total_epoch = 100
batch_size = 100
learning_rate = 0.002

In [0]:
X = tf.placeholder(tf.float32, [None, n_input])
Z = tf.placeholder(tf.float32, [None, n_noise])

## 생성자, 판별자 신경망 변수
* W
* b

In [0]:
## 생성망
# 입력층 -> 은닉층
G_W1 = tf.Variable(tf.random_normal([n_noise, n_hidden], stddev=0.01))
G_b1 = tf.Variable(tf.zeros(n_hidden))

# 은닉층 -> 출력층 
G_W2 = tf.Variable(tf.random_normal([n_hidden, n_input], stddev=0.01))
G_b2 = tf.Variable(tf.zeros([n_input]))

In [0]:
## 판별망
# 입력층 -> 은닉층
D_W1 = tf.Variable(tf.random_normal([n_input, n_hidden], stddev=0.01))
D_b1 = tf.Variable(tf.random_normal([n_hidden]))

# 은닉층 -> 출력층 
D_W2 = tf.Variable(tf.random_normal([n_hidden, 1], stddev=0.01))
D_b2 = tf.Variable(tf.random_normal([1]))

## 노이즈 데이터 발생을 위한 노이즈 생성 함수
* def get_noise(데이터 개수,  노이즈 개수)

In [0]:
def get_noise(batch_size, n_noise):
  return np.random.normal(size=(batch_size, n_noise))

## 생성자와 판별자의 신경망 구성

In [0]:
# 생성자
def generator(noise_z):
  hidden = tf.nn.relu(tf.matmul(noise_z, G_W1) + G_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, G_W2) + G_b2)
  return output

In [0]:
# 판별자
def discriminator(inputs):
  hidden = tf.nn.relu(tf.matmul(inputs, D_W1) + D_b1)
  output = tf.nn.sigmoid(tf.matmul(hidden, D_W2) + D_b2)
  return output

In [0]:
# 생성자 신경망
G = generator(Z)
D_fake = discriminator(G)
D_real = discriminator(X)

## 비용함수, 최적화 함수

In [0]:
# 경찰관의 성능 -> 최대화
loss_D = tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))

# 위조지폐범의 성능 -> 최대화
loss_G = tf.reduce_mean(tf.log(D_fake))

In [0]:
# 사용하는 변수 리스트화
D_var_list = [D_W1, D_b1, D_W2, D_b2]
G_var_list = [G_W1, G_b1, G_W2, G_b2]

In [0]:
# loss_D, loss_G 최대화
train_D = tf.train.AdamOptimizer(learning_rate).minimize(-loss_D, var_list=D_var_list)
train_G = tf.train.AdamOptimizer(learning_rate).minimize(-loss_G, var_list=G_var_list)


In [0]:
# 모델 학습
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(mnist.train.num_examples/batch_size)
loss_val_D, loss_val_G = 0, 0

In [15]:
for epoch in range(20):
  for i in range(total_batch):
    batch_xs, batch_ys = mnist.train.next_batch(batch_size) # 판별망의 데이터 
    noise = get_noise(batch_size, n_noise) # 생성망에 들어가는 입력
    
    _, loss_val_D = sess.run([train_D, loss_D], 
                             feed_dict={X:batch_xs, Z:noise})
    _, loss_val_G = sess.run([train_G, loss_G],
                             feed_dict={Z:noise})
    

  if epoch == 0 or (epoch + 1) % 10 == 0:
    sample_size = 10
    noise = get_noise(sample_size, n_noise) # 노이즈 생성.
    samples = sess.run(G, feed_dict={Z: noise})
    fig, ax = plt.subplots(1, sample_size, figsize=(sample_size, 1))
    
    for i in range(sample_size):
      ax[i].set_axis_off()
      ax[i].imshow(np.reshape(samples[i], (28, 28)))
      
    plt.savefig('samples/{}.png'.format(str(epoch).zfill(3)), bbox_inches='tight')
    plt.close(fig)

  print("Epoch : {}, 판별망의 성능 : {:.4f}, 생성망의 성능 : {:.4f}".format(epoch, 
                                                           loss_val_D,
                                                           loss_val_G))
    
print("최적화 완료!")

/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:395: UserWarning: Warning: converting a masked element to nan.
  dv = (np.float64(self.norm.vmax) -
/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:396: UserWarning: Warning: converting a masked element to nan.
  np.float64(self.norm.vmin))
/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:403: UserWarning: Warning: converting a masked element to nan.
  a_min = np.float64(newmin)
/usr/local/lib/python3.6/dist-packages/matplotlib/image.py:408: UserWarning: Warning: converting a masked element to nan.
  a_max = np.float64(newmax)
/usr/local/lib/python3.6/dist-packages/matplotlib/colors.py:918: UserWarning: Warning: converting a masked element to nan.
  dtype = np.min_scalar_type(value)
/usr/local/lib/python3.6/dist-packages/numpy/ma/core.py:713: UserWarning: Warning: converting a masked element to nan.
  data = np.array(a, copy=False, subok=subok)


Epoch : 0, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 1, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 2, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 3, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 4, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 5, 판별망의 성능 : nan, 생성망의 성능 : nan
Epoch : 6, 판별망의 성능 : nan, 생성망의 성능 : nan


KeyboardInterrupt: ignored